In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft
from scipy.io.wavfile import write
import pyaudio
from matplotlib import cm
from matplotlib.patches import Circle

cmap = cm.get_cmap('plasma')

%matplotlib qt


fig, ax = plt.subplots()

p = pyaudio.PyAudio()

chunk = 512
format = pyaudio.paInt16  
channels = 1  
rate = 10000
duration = 2
numChunks = int(rate / chunk * duration)
chunks = np.array(range(0, numChunks))
stream = p.open(format=format, channels=channels, rate=rate, input=True, frames_per_buffer=chunk)
numSamples = len(chunks)*chunk

samples = np.linspace(0, duration, num = numSamples)

import librosa

def decompose(F):
    m  = librosa.hz_to_midi(F)
    m0 = np.floor(m).astype(int)
    m1 = m0 + 1
    f0 = librosa.midi_to_hz(m0)
    f1 = librosa.midi_to_hz(m1)
    β  = (F - f0) / (f1 - f0)
    return {
        'alpha': 1 - β,
        'beta':  β,
        'note1': librosa.midi_to_note(m0),
        'note2': librosa.midi_to_note(m1)
    }


def noteToColor(note):
    color = 0
    denom = 11
    colorNumber = 0
    if note == "C":
        colorNumber = 0/denom

    elif note == "C#":
        colorNumber = 1/denom

    elif note == "D":
        colorNumber = 2/denom

    elif note == "D#":
        colorNumber = 3/denom
    
    elif note == "E":
        colorNumber = 4/denom

    elif note == "F":
        colorNumber = 5/denom

    elif note == "F#":
        colorNumber = 6/denom

    elif note == "G":
        colorNumber = 7/denom

    elif note == "G#":
        colorNumber = 8/denom

    elif note == "A":
        colorNumber = 9/denom

    elif note == "A#":
        colorNumber = 10/denom

    elif note == "B":
        colorNumber = 11/denom

    
    


def freqToColor(frequency):
    '''So this will take frequency to note then note to color
    we find distance between two notes, note 1 and note 2. The freq is in between two notes 
    alpha * note 1 freq + beta * note 2 freq = the frequency'''
    return decompose(frequency)



i = 0

low = 200
med = 2000
maxR = -1
cir = Circle((0,0), fill = True)
ax.add_patch(cir)
def update(frame):
    global i, intData, colorList, maxR, cir
    
    chunkHere = np.frombuffer(stream.read(chunk), dtype=np.int16)

    
    fourier = fft(chunkHere)
    mag = abs(fourier[:int(len(fourier)/2)])
    indexOfMax = np.argmax(mag)
    frequencies = np.fft.fftfreq(chunk, d=1/rate)[:int(len(fourier)/2)]
    domFreq = frequencies[indexOfMax]
    if domFreq < 20:
        domFreq = 20
    elif domFreq > 20000:
        domFreq = 20000
    domFreqNorm = domFreq / (.5*rate)
    colr = cmap(domFreqNorm)

    cir.set_color(colr)
    #cir.set_radius(mag[indexOfMax])
   
    
    
    
print(freqToColor(400))


#anim = animation.FuncAnimation(fig = fig, func = update, interval = 1)

plt.show()


C:\Users\jshen\AppData\Local\Temp\ipykernel_2500\3893628139.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('plasma')


{'alpha': 0.6565929819557095, 'beta': 0.34340701804429047, 'note1': 'G4', 'note2': 'G♯4'}
